# 1. import packages 导入包

In [ ]:
import torch
import torch.nn as nn #Access neural network layers and modules
import torch.nn.functional as F #Use functional operations like activations without defining layers
from torch.utils.data import Dataset #Base class to create custom dataset
from torch.utils.data import DataLoader #Efficiently load and batch data for training
from dataclasses import dataclass #Create lightweight classes for configuration or parameter

torch.manual_seed(1024)


# 2. Define parameters for GPT 定义GPT的一些参数

In [ ]:
@dataclass #Auto-generates __init__ and __repr__ for easy parameter management
class GPTConfig:
    block_size: int = 512 #文本最大长度 max sequence
    batch_size: int = 12 #Number of samples per training batch
    n_layer: int = 12 #Number of Transformer layers
    n_head: int = 12 #Number of attention heads in multi-head attention
    n_embd: int = 768 #Token embedding dimension (length of vector)
    hidden_dim: int = n_embd #Feed-forward network hidden dimension
    dropout: float = 0.1 #Feed-forward network hidden dimension
    head_size: int = n_embd // n_head #Dimension per attention head
    # follow gpt2 tokenzier
    vocab_size: int = 50257


# 3. Define the Structure of GPT

In [ ]:
# 1. single head attention
class SingleHeadAttention(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.key = nn.Linear(config.hidden_dim, config.head_size)
        self.query = nn.Linear(config.hidden_dim, config.head_size)
        #attention_mask通过register_buffer注册
        #不用计算梯度，节约内存显寸，速度更快
        self.register_buff(
            "attention_mask",
            # tril 下三角
            # block_size 512
            torch.tril(
                torch.ones(config.block_size, config.block_size)
            )
        )
        self.dropout = nn.Dropout(config.dropout)

    def forward(self,x):
        batch_size, seq_len, hidden_dim =  x.size()
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        weight = q @ k.transpose(-2,-1) # @是torch.matmul的简化写法
        





